# LaCATHODE Walkthrough

Just like the CATHODE walkthtough, this is a simple conceptual guide through how the [LaCATHODE method](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.107.114012) for anomaly detection works. This demo assumes that `demos/cathode_walkthrough.ipynb` is understood. The notebook is oversimplified such that it does not make use of all optimization measures implemented in the main paper. It rather shows the core concept while hiding the technical implementation details behind a scikit-learn style API.

Our main concern in `demos/cathode_walkthrough.ipynb` was to get high sensitivity for a new physics signal without actually using the signal-vs-background truth labels in the training. We measured that sensitivity via SIC curves, which do use that truth information. In a real experiment, one would need to extract the signal via some background estimation method. One way, that goes well with weak supervision, is a bump hunt: we select the most anomalous events, then fit a background shape to the resonant feature in the sidebands and compare this estimated background within the signal region to the measured data to see if there is an excess.

Such a bump hunt background estimation works best if the background shape is smooth, even after applying our anomaly selection. If our method sculpts artificial bumps into the signal region, we will have to face additional obstacles, such as finding a more complicated background function and/or modeling the systematic uncertainties from this sculpting, which in turn reduces our signal sensitivity. In this demo, we look at how correlated input features can lead to sculpting in CATHODE and how the LaCATHODE modification can help to avoid this.

In [1]:
!pip install vector scikit-learn==1.4.0

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import sys

from os.path import exists, join, dirname, realpath
from sklearn.metrics import roc_curve
from sklearn.neighbors import KernelDensity
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# adding parent directory to path
parent_dir = dirname(realpath(globals()["_dh"][0]))
sys.path.append(parent_dir)

from sk_cathode.generative_models.conditional_normalizing_flow_torch import ConditionalNormalizingFlow
from sk_cathode.classifier_models.neural_network_classifier import NeuralNetworkClassifier
from sk_cathode.utils.preprocessing import LogitScaler

In [3]:
# :sunglasses:
plt.style.use('dark_background')

Again, we use input data preprocessed via another script `demos/utils/data_preparation.py`. However, this time we add another feature, which is the angular distange between the two leading jets $\Delta R_{jj}$. Thus, it downloads the LHCO R\&D dataset and applies the preprocessing to extract the conditional feature $m=m_{jj}$ and **five** auxiliary features $x=(m_{j1}, \Delta m_{jj}, \tau_{21,j1}, \tau_{21,j2}, \Delta R_{jj})$. Moreover, it divides the $m$ spectrum into signal region (SR) and sideband (SB), and splits the data into training/validation/test sets.

In [4]:
data_path = "/global/cfs/cdirs/ntrain1/anomaly/input_data_deltaR/"

In [5]:
# data preparation (download and high-level preprocessing)
if not exists(join(data_path, "innerdata_test.npy")):
    process = subprocess.run(f"{sys.executable} {join(parent_dir, 'demos', 'utils', 'data_preparation.py')} --outdir {data_path} --add_deltaR", shell=True, check=True)


In [6]:
# data loading
outerdata_train = np.load(join(data_path, "outerdata_train.npy"))
outerdata_val = np.load(join(data_path, "outerdata_val.npy"))
outerdata_test = np.load(join(data_path, "outerdata_test.npy"))
innerdata_train = np.load(join(data_path, "innerdata_train.npy"))
innerdata_val = np.load(join(data_path, "innerdata_val.npy"))
innerdata_test = np.load(join(data_path, "innerdata_test.npy"))

We first start by training CATHODE as before. I.e. we train a conditonal normalizing flow on SB data, sample background-like SR events and train a classifier to distinguish them from "real" SR data.

In [7]:
# either train new flow model from scratch

# We streamline the preprocessing with an sklearn pipeline.
# Ideally we would wrap the whole model, including the flow. But out of the box,
# the pipeline class does not wrap sample() and predict_log_proba() :(
outer_scaler = make_pipeline(LogitScaler(), StandardScaler())

m_train = outerdata_train[:, 0:1]
X_train = outer_scaler.fit_transform(outerdata_train[:, 1:-1])
m_val = outerdata_val[:, 0:1]
X_val = outer_scaler.transform(outerdata_val[:, 1:-1])

flow_savedir = "./trained_flows_deltaR/"
# Let's protect ourselves from accidentally overwriting a trained model.
if not exists(join(flow_savedir, "DE_models")):
    flow_model = ConditionalNormalizingFlow(save_path=flow_savedir,
                                            num_inputs=outerdata_train[:, 1:-1].shape[1],
                                            early_stopping=True, epochs=None,
                                            verbose=True)
    flow_model.fit(X_train, m_train, X_val, m_val)
else:
    print(f"The model exists already in {flow_savedir}. Remove first if you want to overwrite.")

ConditionalNormalizingFlow has 274800 parameters
n_nans = 0
n_highs = 0
n_nans = 0
n_highs = 0

Epoch: 0


Train, Log likelihood in nats: -5.458252:   7%|▋         | 29952/439060 [00:04<01:06, 6111.12it/s]

KeyboardInterrupt: 

In [ ]:
# or loading existing flow model

outer_scaler = make_pipeline(LogitScaler(), StandardScaler())
outer_scaler.fit(outerdata_train[:, 1:-1])

flow_savedir = "./trained_flows_deltaR/"
flow_model = ConditionalNormalizingFlow(save_path=flow_savedir,
                                        num_inputs=outerdata_train[:, 1:-1].shape[1],
                                        load=True)

In [ ]:
# fitting a KDE for the mass distribution based on the inner training set

# we also perform a logit first to stretch out the hard boundaries
m_scaler = LogitScaler(epsilon=1e-8)
m_train = m_scaler.fit_transform(innerdata_train[:, 0:1])

kde_model = KernelDensity(bandwidth=0.01, kernel='gaussian')
kde_model.fit(m_train)

# now let's sample 4x the number of training data
m_samples = kde_model.sample(4*len(m_train)).astype(np.float32)
m_samples = m_scaler.inverse_transform(m_samples)

# drawing samples from the flow model with the KDE samples as conditional
X_samples = flow_model.sample(n_samples=len(m_samples), m=m_samples)

X_samples = outer_scaler.inverse_transform(X_samples)

# assigning "signal" label 0 to samples
samples = np.hstack([m_samples, X_samples, np.zeros((m_samples.shape[0], 1))])

In [ ]:
# comparing samples to inner background (idealized sanity check)

for i in range(innerdata_test[:, :-1].shape[1]):
    _, binning, _ = plt.hist(innerdata_test[innerdata_test[:, -1] == 0, i],
                             bins=100, label="data background",
                             density=True, histtype="step")
    _ = plt.hist(samples[:, i],
                 bins=binning, label="sampled background",
                 density=True, histtype="step")
    plt.legend()
    plt.ylim(0, plt.gca().get_ylim()[1] * 1.2)
    plt.xlabel("feature {}".format(i))
    plt.ylabel("counts (norm.)")
    plt.show()

In [ ]:
# assigning "signal" label 1 to data
clsf_train_data = innerdata_train.copy()
clsf_train_data[:, -1] = np.ones_like(clsf_train_data[:, -1])

clsf_val_data = innerdata_val.copy()
clsf_val_data[:, -1] = np.ones_like(clsf_val_data[:, -1])

# then mixing data and samples into train/val sets together proportionally
n_train = len(clsf_train_data)
n_val = len(clsf_val_data)
n_samples_train = int(n_train / (n_train + n_val) * len(samples))
samples_train = samples[:n_samples_train]
samples_val = samples[n_samples_train:]

clsf_train_set = np.vstack([clsf_train_data, samples_train])
clsf_val_set = np.vstack([clsf_val_data, samples_val])
clsf_train_set = shuffle(clsf_train_set, random_state=42)
clsf_val_set = shuffle(clsf_val_set, random_state=42)

In [ ]:
# either train new NN classifier to distinguish between real inner data and samples

# derive scaler parameters on data only, so it stays the same even if we resample
inner_scaler = StandardScaler()
inner_scaler.fit(clsf_train_data[:, 1:-1])

X_train = inner_scaler.transform(clsf_train_set[:, 1:-1])
y_train = clsf_train_set[:, -1]
X_val = inner_scaler.transform(clsf_val_set[:, 1:-1])
y_val = clsf_val_set[:, -1]

classifier_savedir = "./trained_classifiers_deltaR_CATHODE/"
# Let's protect ourselves from accidentally overwriting a trained model.
if not exists(join(classifier_savedir, "CLSF_models")):
    classifier_model = NeuralNetworkClassifier(save_path=classifier_savedir,
                                               n_inputs=X_train.shape[1],
                                               early_stopping=True, epochs=None,
                                               verbose=True)
    classifier_model.fit(X_train, y_train, X_val, y_val)
else:
    print(f"The model exists already in {classifier_savedir}. Remove first if you want to overwrite.")

In [ ]:
# or alternatively load existing classifer model

inner_scaler = StandardScaler()
inner_scaler.fit(clsf_train_data[:, 1:-1])

classifier_savedir = "./trained_classifiers_deltaR_CATHODE/"
classifier_model = NeuralNetworkClassifier(save_path=classifier_savedir,
                                           n_inputs=clsf_train_data[:, 1:-1].shape[1],
                                           load=True)

In [ ]:
# now let's evaluate the signal extraction performance

X_test = inner_scaler.transform(innerdata_test[:, 1:-1])
y_test = innerdata_test[:, -1]

preds_test = classifier_model.predict(X_test)

with np.errstate(divide='ignore', invalid='ignore'):
    fpr, tpr, _ = roc_curve(y_test, preds_test)
    sic = tpr / np.sqrt(fpr)

    random_tpr = np.linspace(0, 1, 300)
    random_sic = random_tpr / np.sqrt(random_tpr)

plt.plot(tpr, sic, label="CATHODE")
plt.plot(random_tpr, random_sic, "w:", label="random")
plt.xlabel("True Positive Rate")
plt.ylabel("Significance Improvement")
plt.legend(loc="upper right")
plt.show()

With an extra auxiliary feature added to CATHODE we still seem to be doing well. There is a tendency towards slightly lower SIC compared to without, because our generative model has to learn an extra dimension and the classifier is exposed to more noise, in this case where the extra feature does not really help distinguishing this particular signal from background.

But our main focus now is how the background would look like once we apply our anomaly selection. To check this, we evaluate the CATHODE classifier on the full test background (both signal region and sideband) and select the most anomalous 1% of events.

In [ ]:
# let's look at the predictions on the full SR+SB test background
# (also idealized as IRL we cannot isolate from the signal)
fulldata_test = np.vstack([innerdata_test, outerdata_test])
fullbkg_test = fulldata_test[fulldata_test[:, -1] == 0]
fullpreds_test = classifier_model.predict(inner_scaler.transform(fullbkg_test[:, 1:-1])).flatten()

# let's select the 1% most anomalous events
threshold = np.percentile(fullpreds_test, 99)
anomdata_test = fullbkg_test[fullpreds_test > threshold]

# and plot the dijet mass distribution before/after the corresponding cut
_, binning, _ = plt.hist(fullbkg_test[:, 0], bins=100, label="full data", histtype="step")
_ = plt.hist(anomdata_test[:, 0], bins=binning, label="most anomalous data", histtype="step")
plt.axvline(innerdata_test[:, 0].min(), color=plt.rcParams["text.color"], linestyle=":", label="SR")
plt.axvline(innerdata_test[:, 0].max(), color=plt.rcParams["text.color"], linestyle=":")
plt.xlabel("conditional feature")
plt.ylabel("counts")
plt.yscale("log")
plt.legend(loc="upper right")
plt.show()

The result here can look quite different from run to run (which is an issue on its own), but in most cases one sees a visible change of background shape after selecting on the anomaly score. Even worse, there tend to be bumps appearing randomly, even in the signal region, which is a real headache for a background estimation such as the bump hunt, which relies on fitting a smooth background function to extract a signal bump on top.

One can check and see that the behavior is not as dramatic on the original feature set (in `demos/cathode_walkthrough.ipynb`) without $\Delta R_{jj}$. So what is special about this feature? One can get an idea by plotting its distribution separately in SR and SB.

In [ ]:
lowerdata_train = outerdata_train[outerdata_train[:, 0] < innerdata_train[:, 0].min()]
upperdata_train = outerdata_train[outerdata_train[:, 0] > innerdata_train[:, 0].max()]

for i in range(innerdata_train[:, :-1].shape[1]):
    # computing the binning on full outer data
    _, binning = np.histogram(outerdata_train[:, i], bins=100)
    _ = plt.hist(innerdata_train[innerdata_train[:, -1] == 0, i],
                 bins=binning, label="SR background",
                 density=True if i >0 else False, histtype="step")
    _ = plt.hist(lowerdata_train[lowerdata_train[:, -1] == 0, i],
                 bins=binning, label="lower SB background",
                 density=True if i >0 else False, histtype="step")
    _ = plt.hist(upperdata_train[upperdata_train[:, -1] == 0, i],
                 bins=binning, label="upper SB background",
                 density=True if i >0 else False, histtype="step")
    plt.legend()
    plt.ylim(0, plt.gca().get_ylim()[1] * 1.2)
    plt.xlabel("feature {}".format(i))
    plt.ylabel("counts (norm.)" if i > 0 else "counts")
    plt.show()

So while the first four auxiliary features (feature 1-4) seem quite similar in SR and SB, there is a siginificant difference between the three regions in $\Delta R_{jj}$ (feature 5). So there are clearly strong correlations between the input space of our neural network classifier and the resonant feature. Thus, the classifier will learn this dependence as well and our anomaly score inherits this correlation. Cutting on this anomaly score thus translates to some funny cut on the resonant feature. Looking at the plot above, we see that we even extrapolate the classifier into regions (SB) outside its training space (SR). Neural networks are not known to handle this type of extrapolation well.

We could either try to remove this correlation a-posteriori, or we could remove the correlation from the input features *before* the classifier training. The latter is something that we actually aready have lying around just from CATHODE. We trained a conditional normalizing flow, which is a function $f(x, m)$ that maps data space $x$ to the latent space $z$, which follows a standard normal distribution, and it does so continuously for every $m$. Thus, the $z$ and $m$ will be effectively decorrelated.

This is what we try to make use of in latent CATHODE (LaCATHODE). Before we train the CATHODE classifier, we move all the SR training data to the latent space using the learned flow model. The background should just be distributed like a standard gaussian, so the sampling becomes straightforward. Once we want to infer the anomaly score of our test data, we also first move it to the same latent space. SR and SB should be identically distributed there.

Let's do this in practice below:

In [ ]:
# if necessary loading existing flow model again

outer_scaler = make_pipeline(LogitScaler(), StandardScaler())
outer_scaler.fit(outerdata_train[:, 1:-1])

flow_savedir = "./trained_flows_deltaR/"
flow_model = ConditionalNormalizingFlow(save_path=flow_savedir,
                                        num_inputs=outerdata_train[:, 1:-1].shape[1],
                                        load=True)

In [ ]:
# move all the inner training and validation data to the latent space of the flow

latent_train_data = flow_model.transform(outer_scaler.transform(innerdata_train[:, 1:-1]),
                                         m=innerdata_train[:, 0:1])
latent_val_data = flow_model.transform(outer_scaler.transform(innerdata_val[:, 1:-1]),
                                       m=innerdata_val[:, 0:1])

# we know how perfect background samples should like in this space: a standard normal
latent_samples = np.random.randn(4*latent_train_data.shape[0], latent_train_data.shape[1])

In [ ]:
for i in range(latent_train_data.shape[1]):
    _, binning, _ = plt.hist(latent_samples[:, i],
                             bins=100, label="latent sample background",
                             density=True, histtype="step")
    _ = plt.hist(latent_train_data[innerdata_train[:, -1] == 0, i],
                 bins=binning, label="latent data background",
                 density=True, histtype="step")
    plt.legend()
    plt.ylim(0, plt.gca().get_ylim()[1] * 1.2)
    plt.xlabel("latent feature {}".format(i))
    plt.ylabel("counts (norm.)")
    plt.show()

In [ ]:
# assigning "signal" label 1 to data and 0 to samples
clsf_latent_train_data = np.hstack([latent_train_data,
                                    np.ones((latent_train_data.shape[0], 1))])
clsf_latent_val_data = np.hstack([latent_val_data,
                                  np.ones((latent_val_data.shape[0], 1))])
clsf_latent_samples = np.hstack([latent_samples,
                                 np.zeros((latent_samples.shape[0], 1))])

# then mixing data and samples into train/val sets together proportionally
n_train = len(clsf_latent_train_data)
n_val = len(clsf_latent_val_data)
n_samples_train = int(n_train / (n_train + n_val) * len(clsf_latent_samples))
clsf_latent_samples_train = clsf_latent_samples[:n_samples_train]
clsf_latent_samples_val = clsf_latent_samples[n_samples_train:]

clsf_latent_train_set = np.vstack([clsf_latent_train_data, clsf_latent_samples_train])
clsf_latent_val_set = np.vstack([clsf_latent_val_data, clsf_latent_samples_val])
clsf_latent_train_set = shuffle(clsf_latent_train_set, random_state=42)
clsf_latent_val_set = shuffle(clsf_latent_val_set, random_state=42)

In [ ]:
# either train new NN classifier to distinguish between real inner data and samples

# derive scaler parameters on data only, so it stays the same even if we resample
latent_scaler = StandardScaler()
latent_scaler.fit(clsf_latent_train_data[:, :-1])

X_train = latent_scaler.transform(clsf_latent_train_set[:, :-1])
y_train = clsf_latent_train_set[:, -1]
X_val = latent_scaler.transform(clsf_latent_val_set[:, :-1])
y_val = clsf_latent_val_set[:, -1]

latent_classifier_savedir = "./trained_classifiers_deltaR_LaCATHODE/"
# Let's protect ourselves from accidentally overwriting a trained model.
if not exists(join(latent_classifier_savedir, "CLSF_models")):
    latent_classifier_model = NeuralNetworkClassifier(save_path=latent_classifier_savedir,
                                                      n_inputs=X_train.shape[1],
                                                      early_stopping=True, epochs=None,
                                                      verbose=True)
    latent_classifier_model.fit(X_train, y_train, X_val, y_val)
else:
    print(f"The model exists already in {latent_classifier_savedir}. Remove first if you want to overwrite.")

In [ ]:
# or alternatively load existing classifer model

latent_scaler = StandardScaler()
latent_scaler.fit(clsf_latent_train_data[:, :-1])

latent_classifier_savedir = "./trained_classifiers_deltaR_LaCATHODE/"
latent_classifier_model = NeuralNetworkClassifier(save_path=latent_classifier_savedir,
                                                  n_inputs=clsf_latent_train_data[:, :-1].shape[1],
                                                  load=True)

In [ ]:
# We will now first map data through the flow and its preprocessing and then
# through the classifier and its preprocessing to get the final prediction.
# Let's simplify this chain with a single pipeline.
lacathode_predictor = make_pipeline(outer_scaler,
                                    flow_model,
                                    latent_scaler,
                                    latent_classifier_model)

In [ ]:
# now let's again evaluate the signal extraction performance

preds_test = lacathode_predictor.predict(innerdata_test[:, 1:-1],
                                         m=innerdata_test[:, 0:1]
                                         ).flatten()
y_test = innerdata_test[:, -1]

# clean out potential NaNs
preds_test_clean = preds_test[~np.isnan(preds_test)]
y_test_clean = y_test[~np.isnan(preds_test)]

with np.errstate(divide='ignore', invalid='ignore'):
    fpr, tpr, _ = roc_curve(y_test_clean, preds_test_clean)
    sic = tpr / np.sqrt(fpr)

    random_tpr = np.linspace(0, 1, 300)
    random_sic = random_tpr / np.sqrt(random_tpr)

plt.plot(tpr, sic, label="LaCATHODE")
plt.plot(random_tpr, random_sic, "w:", label="random")
plt.xlabel("True Positive Rate")
plt.ylabel("Significance Improvement")
plt.legend(loc="upper right")
plt.show()

The signal extraction performance in terms of SIC tends to be a bit lower for LaCATHODE than default CATHODE, but remains still very non-trivial for this type of signal. But remember that we would still need a background estimation procedure to extract the signal in a real experiment. For that let's see how smooth the background looks like after selecting on the anomaly score.

In [ ]:
# and plot again the dijet mass distribution before/after the corresponding cut

# let's look at the predictions on the full SR+SB test data
fulldata_test = np.vstack([innerdata_test, outerdata_test])
fullbkg_test = fulldata_test[fulldata_test[:, -1] == 0]
fullpreds_latent_test = lacathode_predictor.predict(fullbkg_test[:, 1:-1],
                                                    m=fullbkg_test[:, 0:1]
                                                    ).flatten()

# let's select the 1% most anomalous events
threshold = np.percentile(fullpreds_latent_test[~np.isnan(fullpreds_latent_test)], 99)
anomdata_latent_test = fullbkg_test[fullpreds_latent_test > threshold]

# and plot the dijet mass distribution before/after the corresponding cut
_, binning, _ = plt.hist(fullbkg_test[:, 0], bins=100, label="full data", histtype="step")
_ = plt.hist(anomdata_latent_test[:, 0], bins=binning, label="most anomalous data", histtype="step")
plt.axvline(innerdata_test[:, 0].min(), color=plt.rcParams["text.color"], linestyle=":", label="SR")
plt.axvline(innerdata_test[:, 0].max(), color=plt.rcParams["text.color"], linestyle=":")
plt.xlabel("conditional feature")
plt.ylabel("counts")
plt.yscale("log")
plt.legend(loc="upper right")
plt.show()

The plot above should usually look much better than with default CATHODE. As the normalizing flow removes the correlation between the training features and the resonant one in the latent space, we also do not expect the output of the classifier to depend on the resonant feature anymore and does not sculpt the background there. Fitting the above background from the sideband with a smooth function and comparing to the data in the signal region should thus be much easier.